In [16]:
!pip install openai==0.28 

In [18]:
import pandas as pd 
df = pd.read_csv('Mobile Dataset1.csv')
df.head()

,Model Name,RAM,ROM,Main Cam,Front Cam,Battery,Screen Size,Processor,OS,Price,Description,mobile_feature
0,Redmi 10,4 GB,64 GB,50 MP,5 MP,6000 mAh,(6.7 inch) HD+ Display,Qualcomm Snapdragon 680 Processor,Android,"10,000",Let go of the inhibitions and enjoy speed oper...,NaN
1,Poco M4 Pro,6 GB,128 GB,64 MP,16 MP,5000 mAh,(6.43 inch) Full HD+ AMOLED Display,Mediatek Helio G96 Processor,Android,"15,000","With 6 GB of RAM, 64 GB of internal storage, a...",NaN
2,9 Pro Plus,8 GB,128 GB,50 MP + 2 MP,16 MP,4500 mAh,(6.4 inch) Full HD+ AMOLED Display,Mediatek Dimensity 920 Processor,Android,"20,000",Display your personality and make a fashion st...,NaN
3,10 Pro+ 5G,8 GB,128 GB,108MP + 8MP,16 MP,5000 mAh,(6.7 inch) Full HD+ Display,Mediatek Dimensity 1080 5G Processor,Android,"25,000",Display your personality and make a fashion st...,NaN
4,Redmi Note 11 Pro,8 GB,128 GB,108 MP,20 MP,5000 mAh,(6.67 inch) Full HD+ Display,Mediatek Helio G96,Android,"17,000","Using the REDMI Note 11 Pro Smartphone, you ca...",NaN


In [19]:
import os,json,ast
import openai
import pandas as pd

In [20]:
# read the API key
openai.api_key = open("OpenAI_API_Key.txt", "r").read().strip()

In [21]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''
    delimiter = "####"
    example_user_request = { 'Display Size': 'high', 'Battery Life': 'high', 'Performance': 'high', 'Budget': '20000'}
    
    system_message = f"""
    You are an intelliegent Mobile Expert and your goal is to suggest the best Mobile to user.
    You need to ask relevant questions and understand user profile and requirement. 
    You final objective is to fill the values for the different keys ('Display Size','Battery Life', 'Performance', 'Budget') in the Python Dictonary and be confident values.
    These key value pairs define the user's profile.
    The Python dictionay looks like this {{'Display Size': 'values', 'Battery Life': 'values', 'Performance': 'values', 'Budget': 'values' }}
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as defined by user.
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values currently in the dictionary are only representative values. 
    
    {delimiter}Here are some instruction for values of the dieffernt keys. You must follow this, you'll be heavily penalized
    - The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as defined by user.
    - The value for 'Budget' should be numerical value extracted from ther user's response. 
    - 'Budget' value needs to be greater than or equal to 7999 INR. If user says less than that, please mention that there are no Mobiles available in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response 
    {delimiter}
    
    To fill the Dictionary, you need to have following chain of thoughts:
    {delimiter} Thought 1 : Ask question to understand the user's profile and requirement. \n
    If their primary use for Mobile is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('Display Size', 'Battery Life', 'Performance', 'Budget') in the python dictionary by understanding the user requirements.
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the questions to capture the requirements \n {delimiter}
    
    {delimiter}Thought 2 : Now you are trying to fill the values of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all they keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for keys and are confident about the same. 
    If yes, mare to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed directly citing the key you want to understand value for. {delimiter}
    
    {delimiter}Thought 3 : Check if you have correctly updated the values for the different keys in the python dictionary. 
    If you are not confident about any of the values, ask clarifyinf questions. {delimiter}
    
    Follow the above chain of thoughts and only output the final updated python dictionary. \n
    
    {delimiter}Here is a sample conversation between the user and assistant :
    User : "I'm looking for a Mobile phone."
    Assistant : "Hello I'm your Mobile Assistant tell my what are your requirements and in what budget you are looking a buy a phone."
    User : "I am looking for camera centric mobile phone."
    Assistant : "Soo, it must have a good battery to run for a long day and a good processor to process the images shot from camera."
    User : "Yes."
    Assistant : "Is your nessicity that you should have a bigger display or any specific type of displays ? "
    User : "Yaa I need Amoled or HD+ Display "
    Assistant : "Cool Could you kindly let me know your budget for the mobile phone? This will help me find option that fit within your price range while meeting the specified requirements."
    User : "My budget is Around 30k. "
    
    Assistant : "{example_user_request}"
    {delimiter}
    
    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [9]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelliegent Mobile Expert and your goal is to suggest the best Mobile to user.\n    You need to ask relevant questions and understand user profile and requirement. \n    You final objective is to fill the values for the different keys (\'Display Size\',\'Battery Life\', \'Performance\', \'Budget\') in the Python Dictonary and be confident values.\n    These key value pairs define the user\'s profile.\n    The Python dictionay looks like this {\'Display Size\': \'values\', \'Battery Life\': \'values\', \'Performance\': \'values\', \'Budget\': \'values\' }\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as defined by user.\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values currently in the dictionary are only representative values. \n    \n    ####Here are some instruction for values of

In [10]:
def get_chat_model_completions(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7, # this is the degree of randomness of the model's output
        max_tokens = 2500
    )
    return response.choices[0].message["content"]

In [42]:
debug_introduction = get_chat_model_completions(debug_conversation)
print(debug_introduction)

Hello! I'm your Mobile Expert Assistant. Please let me know your requirements and budget, and I'll suggest the best mobile phone for you.


In [43]:
debug_user_input = "Hi, I am Vishal, I need a Mobile for which has good camera."

In [44]:
debug_conversation.append({"role" :  "user", "content" : debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Hello Vishal! I'm your Mobile Expert. I can help you find a mobile phone with a good camera. Could you please tell me if you have any other specific requirements for the phone?


#### Adding Moderation check 

In [11]:
def moderation_check(user_input) :
    response = openai.Moderation.create(input = user_input)
    moderation_output = response["results"][0]
    if moderation_output["flagged"] == True :
        return "Flagged"
    else: 
        return "Not Flagged"

Checking moderation the debug_input user 

In [46]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


In [23]:

print(moderation_check("I want to kill Ravan."))
print(moderation_check("I need a mobile to hack Android."))

Flagged
Not Flagged


In [12]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior evluator who has an eye for detail.
    Yor are provided an input. You naeed to evaluate if the input has the following keys: 'Display Size', 'Battery Life', 'Performance', 'Budget'
    Next you need to evaluate if the keys have the value filled correctly.
    The values for all keys, except 'budget' needa to contain a number with currency. 
    Output a string 'Yes' is the input contains the dictionary with the values correctly filled for all keys. 
    Otherwise out the string 'No'. 
    
    Here is the input : {response_assistant}
    Only output a one-word string - Yes/No. 
    """
    confirmation = openai.Completion.create(
                                    model = "gpt-3.5-turbo-instruct",
                                    prompt = prompt,
                                    temperature = 0)
    
    return confirmation["choices"][0]["text"]

In [86]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)


No


In [13]:
debug_response_assistant_n = f"""

Based on the information you have provided, I have updated the values in the dictionary as follows:

{{'Display Size': 'low', 'Battery Life': 'high', 'Performance': 'low', 'Budget': '25000'}}

Please note that the values for 'Display Size' is set to 'low' since they are not a priority for your camera sentric mobile phone.

Is there anything else I can assist you with?
"""

In [60]:
intent_confirmatio_layer(debug_response_assistant_n)

'\n\nNo'

In [22]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'Display Size': 'low','Battery Life': 'high','Performance': 'high','Budget': '25000 INR'}
    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract and return only the python dictionary from the input.
            The output should match the format as {user_req}.
            The output should contain the exact keys and values as present in the input.

            Here are some sample input output pairs for better understanding:
            {delimiter}
            input: - Display Size : low - Battery Life : high - Performance : high - Budget: 20,000 INR
            output: {{'Display Size': 'low', 'Battery Life': 'high', 'Performance': 'high', 'Budget': '20000'}}

            input: {{'Display Size':     'low', 'Battery Life':     'high', 'Performance':    'low', 'Budget': '10,000'}}
            output: {{'Display Size': 'low', 'Battery Life': 'high', 'Performance': 'low', 'Budget': '10000'}}

            input: Here is your user profile 'Display Size': 'high','Battery Life': 'medium','Perdormance': 'medium','Budget': '17000 INR'
            output: {{'Display Size': 'high','Battery Life': 'medium','Performance': 'medium', Budget': '17000'}}
            {delimiter}

            Here is the input {response}

            """
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens = 2000,
        temperature=0.4,
        #top_p=0.4
    )
    return response["choices"][0]["text"]

In [23]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

In [24]:
debug_response_assistant_n = f"""Thank you for providing your budget. Based on your budget of 20,000 INR, I will consider this while recommending suitable Mobile Phone options for you.
Here is the final recommendation for your Mobile Phone:

- Display Size: low
- Battery Life: high
- Performance: high
- Budget: 20,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget. Let me know if there's anything else I can assist you with!"""

In [70]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)


{'Display Size': 'low', 'Battery Life': 'high', 'Performance': 'high', 'Budget': '20000'}


In [25]:
debug_conversation = initialize_conversation()
debug_introduction = get_chat_model_completions(debug_conversation)
debug_user_input = "Hi, I am Vishal, i am looking for camera centiric mobile phone."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
response_dict_n = dictionary_present(debug_response_assistant_n)

In [28]:
def product_map_layer(mobile_description):

    delimiter = "#####"
    mobile_spec = {
        "Display Size": "(Screen Size in inches)",
        "Battery Life": "(Battery capacity in mAh)",
        "Performance": "(Processor, RAM)",
        "Budget": "(Price range)"
    }

    values = {'low', 'medium', 'high'}

    prompt = f"""
    You are a Mobile Specifications Classifier whose task is to extract key features from mobile descriptions.
    To analyze each mobile, follow these steps:
    Step 1: Extract the mobile's primary features from the description: {mobile_description}
    Step 2: Store the extracted features in {mobile_spec}
    Step 3: Classify each of the items in {mobile_spec} into {values} based on the following rules:
    {delimiter}
    Display Size: low: < if screen size is below 6.2 inches >,
    medium: < if screen size is between 6.2 and 6.5 inches >,
    high: < if screen size is above 6.5 inches >,

    Battery Life: low: < if battery capacity is below 4700 mAh >,
    medium: < if battery capacity is between 5000 and 5500 mAh >,
    high: < if battery capacity is above 5600 mAh >,

    Performance: low: < if entry-level processor and 4GB RAM >,
    medium: < if mid-range processor and 6GB RAM >,
    high: < if high-performance processor and 6-8GB RAM or more >,

    {delimiter}

    {delimiter}
    Here are some input-output pairs for few-shot learning:
    input1: "The Samsung Galaxy A30s features a 6.4-inch Super AMOLED display providing vibrant colors and sharp visuals. With a battery capacity of 4000 mAh, it offers long-lasting usage. It is powered by an Exynos 7904 processor and 4GB RAM, ensuring smooth performance for daily tasks. Priced at 15,000/- INR it's a mid-range device suitable for budget-conscious users."
    output1: {{'Display Size': 'medium', 'Battery Life': 'low', 'Performance': 'low'}}

    input2: "The Apple iPhone 13 Pro Max boasts a 6.7-inch OLED display offering stunning visuals. It houses a 4352 mAh battery providing extensive usage. Equipped with an A15 Bionic chip and 8GB RAM, it delivers top-notch performance. Priced at 60,000/- INR it's a premium device catering to users seeking high-end features."
    output2: {{'Display Size': 'high', 'Battery Life': 'low', 'Performance': 'high'}}
    {delimiter}

    Follow the above steps and output the dictionary {mobile_spec} for the given mobile description: {mobile_description}.
    """

    # Using the Completion endpoint
    response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=2000,
        temperature=0.3,
        # top_p=0.4
    )
    return response["choices"][0]["text"]


In [31]:

mobile_df= pd.read_csv('Mobile Dataset1.csv')

mobile_df['mobile_feature'] = mobile_df['Description'].apply(lambda x: product_map_layer(x))

In [32]:
mobile_df.to_csv("updated_mobile.csv", index=False, header=True)

In [30]:
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

In [113]:
string = " \n \t           Output: {{'Display Size': 'high', 'Battery Life': 'high', 'Performance': 'low', 'Budget': '20000'}}ad"
extracted_dict = extract_dictionary_from_string(string)
print(extracted_dict)

{'display size': 'high', 'battery life': 'high', 'performance': 'low', 'budget': '20000'}


In [114]:
def compare_mobiles_with_user(user_req_string):
    mobile_df = pd.read_csv('updated_mobile.csv') 
    user_requirements = extract_dictionary_from_string(user_req_string)
    budget = int(user_requirements.get('budget', '0').replace(',', '').split()[0])

    filtered_mobiles = mobile_df.copy()
    filtered_mobiles['Price'] = filtered_mobiles['Price'].str.replace(',', '').astype(int)
    filtered_mobiles = filtered_mobiles[filtered_mobiles['Price'] <= budget].copy()

    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }

    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_mobiles['Score'] = 0
    for index, row in filtered_mobiles.iterrows():
        user_product_match_str = row['mobile_feature']
        mobile_values = extract_dictionary_from_string(user_product_match_str)
        score = 0

        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            mobile_value = mobile_values.get(key, None)
            mobile_mapping = mappings.get(mobile_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if mobile_mapping >= user_mapping:
                # If the mobile value is greater than or equal to the user value, the score is incremented by 1
                score += 1

        filtered_mobiles.loc[index, 'Score'] = score

    # Sort the mobiles by score in descending order and return the top 3 products
    top_mobiles = filtered_mobiles.drop('mobile_feature', axis=1)
    top_mobiles = top_mobiles.sort_values('Score', ascending=False).head(3)

    return top_mobiles.to_json(orient='records')

In [115]:
top_3_mobiles = compare_mobiles_with_user(response_dict_n)
print(top_3_mobiles)

[{"Model Name":"9 Pro Plus ","RAM":"8 GB","ROM ":"128 GB","Main Cam ":"50 MP + 2 MP","Front Cam ":"16 MP","Battery ":"4500 mAh","Screen Size ":"(6.4 inch) Full HD+ AMOLED Display","Processor ":"Mediatek Dimensity 920 Processor","OS":"Android","Price":20000,"Description":"Display your personality and make a fashion statement with the Realme 9 Pro+ which gives you a fantastic user experience. With its (6.4) AMOLED 2160 Hz PWM Dimming, TUV Rheinland Low Blue Light Certificate, 1260 Hz Turbocharged Touch Sampling, and X-touch Anti-mistouch Algorithm, this smartphone makes a great advancement. With its 50MP ProLight Camera, this smartphone also provides you with amazing photographic and filming features that let you capture priceless memories. This smartphone further advances the advancement by providing you with a Dimensity 920 Processor, which is completely compatible with all  bands. Moreover, the Realme 9 Pro+ large 8+8 GB Dynamic RAM arrangement allows you to view many files or program

In [119]:
def recommendation_validation(mobile_recommendation):
    data = json.loads(mobile_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1

In [120]:
validated_data = recommendation_validation(top_3_mobiles)
print(validated_data)

[{'Model Name': '9 Pro Plus ', 'RAM': '8 GB', 'ROM ': '128 GB', 'Main Cam ': '50 MP + 2 MP', 'Front Cam ': '16 MP', 'Battery ': '4500 mAh', 'Screen Size ': '(6.4 inch) Full HD+ AMOLED Display', 'Processor ': 'Mediatek Dimensity 920 Processor', 'OS': 'Android', 'Price': 20000, 'Description': 'Display your personality and make a fashion statement with the Realme 9 Pro+ which gives you a fantastic user experience. With its (6.4) AMOLED 2160 Hz PWM Dimming, TUV Rheinland Low Blue Light Certificate, 1260 Hz Turbocharged Touch Sampling, and X-touch Anti-mistouch Algorithm, this smartphone makes a great advancement. With its 50MP ProLight Camera, this smartphone also provides you with amazing photographic and filming features that let you capture priceless memories. This smartphone further advances the advancement by providing you with a Dimensity 920 Processor, which is completely compatible with all  bands. Moreover, the Realme 9 Pro+ large 8+8 GB Dynamic RAM arrangement allows you to view 

In [121]:
len(validated_data)

2

#### Product Recommendation Layer 

In [122]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent mobile gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue: {products}.\
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each mobile in the following format, in decreasing order of price of mobiles:
    1. <Mobile Name> : <Major specifications of the mobile>, <Price in Rs>
    2. <Mobile Name> : <Major specifications of the mobile>, <Price in Rs>

    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [123]:
debug_conversation_reco = initialize_conv_reco(top_3_mobiles)

In [125]:
debug_recommendation = get_chat_model_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

1. Redmi Note 11 Pro: 8 GB RAM, 128 GB ROM, 108 MP Main Camera, 20 MP Front Camera, 5000 mAh Battery, 6.67 inch Full HD+ Display, Mediatek Helio G96 Processor, Android OS, Price: Rs 17,000
2. Realme 9 Pro Plus: 8 GB RAM, 128 GB ROM, 50 MP + 2 MP Main Camera, 16 MP Front Camera, 4500 mAh Battery, 6.4 inch Full HD+ AMOLED Display, Mediatek Dimensity 920 Processor, Android OS, Price: Rs 20,000
3. Redmi 10: 4 GB RAM, 64 GB ROM, 50 MP Main Camera, 5 MP Front Camera, 6000 mAh Battery, 6.7 inch HD+ Display, Qualcomm Snapdragon 680 Processor, Android OS, Price: Rs 10,000



In [126]:
debug_conversation_reco.append({"role": "user", "content": "This is my user profile" + response_dict_n})
debug_conversation_reco.append({"role": "assistant", "content": debug_recommendation})

In [127]:
debug_user_input = "Which one is value for money ?"

In [128]:
debug_conversation_reco.append({"role": "user", "content": debug_user_input})
debug_response_asst_reco = get_chat_model_completions(debug_conversation_reco)
print('\n' + debug_response_asst_reco + '\n')


Based on your budget of Rs 20,000 and considering value for money, the Redmi Note 11 Pro would be a good choice. It offers a higher resolution main camera (108 MP) and a larger battery capacity (5000 mAh) compared to the Realme 9 Pro Plus. Additionally, it is priced at Rs 17,000, which is lower than the Realme 9 Pro Plus.



In [34]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    top_3_mobiles = None
    user_input = ''

    while(user_input != "exit"):
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_mobiles is None:
            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_model_completions(conversation)

            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            moderation = moderation_check(confirmation)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
            else:
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
                response = dictionary_present(response_assistant)

                moderation = moderation_check(response)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                print('\n' + response + '\n')
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_mobiles = compare_mobiles_with_user(response) 

                validated_reco = recommendation_validation(top_3_mobiles)

                if len(validated_reco) == 0:
                    print("Sorry, we do not have mobiles that match your requirements. Connecting you to a human expert.")
                    break

                conversation_reco = initialize_conv_reco(validated_reco)
                recommendation = get_chat_model_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "user", "content": "This is my user profile" + response})

                conversation_reco.append({"role": "assistant", "content": recommendation})

                print(recommendation + '\n')

        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_model_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})
